In [1]:
# Imports for project
import pandas as pd

In [7]:
# Dataset
path= "bank-full.csv"

dataframe = pd.read_csv((path), delimiter=';', quotechar='"')

In [8]:
dataframe.describe() # for numerical data

,age,balance,day,duration,campaign,pdays,previous
count,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000,45211.000000
mean,40.936210,1362.272058,15.806419,258.163080,2.763841,40.197828,0.580323
std,10.618762,3044.765829,8.322476,257.527812,3.098021,100.128746,2.303441
min,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000
25%,33.000000,72.000000,8.000000,103.000000,1.000000,-1.000000,0.000000
50%,39.000000,448.000000,16.000000,180.000000,2.000000,-1.000000,0.000000
75%,48.000000,1428.000000,21.000000,319.000000,3.000000,-1.000000,0.000000
max,95.000000,102127.000000,31.000000,4918.000000,63.000000,871.000000,275.000000


In [10]:
# Finding all the features and categories included
for column in dataframe.columns:
    if dataframe[column].dtype == 'object' or dataframe[column].dtype == 'int64':
        print(f"Feature: {column}")
        print(f"Data Type: {dataframe[column].dtype}")
        
        if dataframe[column].dtype == 'object':  # Check for categorical features
            print(f"Categories: {dataframe[column].unique()}")
        
        print("-" * 50)


Feature: age
Data Type: int64
--------------------------------------------------
Feature: job
Data Type: object
Categories: ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'retired' 'admin.' 'services' 'self-employed' 'unemployed' 'housemaid'
 'student']
--------------------------------------------------
Feature: marital
Data Type: object
Categories: ['married' 'single' 'divorced']
--------------------------------------------------
Feature: education
Data Type: object
Categories: ['tertiary' 'secondary' 'unknown' 'primary']
--------------------------------------------------
Feature: default
Data Type: object
Categories: ['no' 'yes']
--------------------------------------------------
Feature: balance
Data Type: int64
--------------------------------------------------
Feature: housing
Data Type: object
Categories: ['yes' 'no']
--------------------------------------------------
Feature: loan
Data Type: object
Categories: ['no' 'yes']
------------------------------------

**Exploring and preparing the dataset for model implementation**

* Find Unwanted columns
* find missing values
* relation between categorical features and label
* find discrete features
* Find continous features
* find outliers in numerical features
* Check the balance of the dataset based on target values in classification



In [15]:
dataframe.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [14]:
missing_values = dataframe.isnull().sum()

# Filter columns with missing values
features_na = missing_values[missing_values > 0]

# If there are missing values
if not features_na.empty:
    for feature, count in features_na.items():
        missing_percentage = np.round(count / len(df) * 100, 4)
        print(f"{feature}: {missing_percentage} % missing values")
else:
    print("No missing values")

No missing values


In [17]:
for column in dataframe.columns:
    if dataframe[column].dtype == 'object': # Check for categorical features
        num_categories = len(dataframe[column].unique()) 
        print(f"Feature: {column}")
        print(f"Number of Categories: {num_categories}")
        print("-" * 50)


Feature: job
Number of Categories: 12
--------------------------------------------------
Feature: marital
Number of Categories: 3
--------------------------------------------------
Feature: education
Number of Categories: 4
--------------------------------------------------
Feature: default
Number of Categories: 2
--------------------------------------------------
Feature: housing
Number of Categories: 2
--------------------------------------------------
Feature: loan
Number of Categories: 2
--------------------------------------------------
Feature: contact
Number of Categories: 3
--------------------------------------------------
Feature: month
Number of Categories: 12
--------------------------------------------------
Feature: poutcome
Number of Categories: 4
--------------------------------------------------
Feature: y
Number of Categories: 2
--------------------------------------------------
